# Load libraries

In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# image processing
from keras.preprocessing import image as image_utils

# pretrained nets
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop

In [ ]:
train_image_files_path = "/keras2production/fruits/Training/"
valid_image_files_path = "/keras2production/fruits/Test/"

# Keras

https://keras.io/

In [ ]:
fruit_list = ["Apricot", "Avocado", "Banana", "Clementine", "Cocos", "Kiwi", "Lemon", "Limes", 
              "Mandarine", "Orange", "Peach", "Pineapple", "Plum", "Pomegranate", "Raspberry", 
              "Strawberry"]
output_n = len(fruit_list)
img_width = 32
img_height = 32
channels = 3
batch_size = 32

## Pretrained nets

https://keras.io/applications/

### Modify VGG16

- transfer learning (freeze all but the penultimate layer and re-train the last Dense layer) and 
- fine tuning (un-freeze the lower convolutional layers and retrain more layers)

Validation set: fit_generator has no option validation_split

https://keras.io/applications/#usage-examples-for-image-classification-models

In [ ]:
# important: exclude top layers
# we can define our own input_shape, since VGG only provides the learned kernels
base_model = VGG16(weights='imagenet', include_top=False, 
                   input_shape=(img_width, img_height, channels))

In [ ]:
base_model.summary()

In [ ]:
# Freeze all layers
base_model.trainable = False

In [ ]:
train_data_gen = ImageDataGenerator(
    rescale = 1 / 255
)

valid_data_gen = ImageDataGenerator(
    rescale = 1 / 255
)

train_image_array_gen = train_data_gen.flow_from_directory(
    train_image_files_path,
    target_size = (img_width, img_height),
    class_mode = 'categorical',
    classes = fruit_list,
    color_mode = 'rgb', 
    batch_size = batch_size,
    seed = 42)

valid_image_array_gen = valid_data_gen.flow_from_directory(
    valid_image_files_path,
    target_size = (img_width, img_height),
    class_mode = 'categorical',
    classes = fruit_list,
    color_mode = 'rgb', 
    batch_size = batch_size,
    seed = 42)

In [ ]:
# check that the data generator is set up correctly
input_shape = train_image_array_gen.image_shape
classes = train_image_array_gen.class_indices
num_classes = len(classes)
class_counts = np.unique(train_image_array_gen.classes, return_counts=True)[1]
print("Shape:" + str(input_shape))
print("Number of classes:" + str(num_classes))
print("Classes:" + str(classes))

chart = plt.bar(classes.keys(), class_counts)
plt.xticks(rotation=90)
plt.show(chart)

In [ ]:
train_samples = train_image_array_gen.n
valid_samples = valid_image_array_gen.n
print(train_samples, valid_samples)

In [ ]:
# Create the model
model = Sequential()
 
# Add the base model
model.add(base_model)
 
# Add new layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_n, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer = RMSprop(lr = 0.0001, decay = 1e-6),
              metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(
    train_image_array_gen,
    steps_per_epoch = int(train_samples / batch_size), 
    epochs = 3, 
    validation_data = valid_image_array_gen,
    validation_steps = int(valid_samples / batch_size),
    verbose = 1
)

In [ ]:
# Save the weights
model.save_weights('model_pre.h5')

# Save the model architecture
with open('model_pre_architecture.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'lower right')
plt.show()

### Unfreezing more layers, Early Stopping and Model Checkpoints

In [ ]:
base_model.trainable = False
for layer in base_model.layers:
    if layer.name.startswith('block5_conv'):
        layer.trainable = True

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

top_weights_path = 'top_model.h5'
callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor = 'val_acc', 
                    verbose = 1, save_best_only = True),
    EarlyStopping(monitor = 'val_acc', patience = 5, verbose = 1)
]

In [ ]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer = RMSprop(lr = 0.000001, decay = 1e-6),
              metrics = ['accuracy', 'mse'])

In [ ]:
history = model.fit_generator(
      train_image_array_gen,
      steps_per_epoch = int(train_samples / batch_size),
      epochs = 5,
      validation_data = valid_image_array_gen,
      validation_steps = int(valid_samples / batch_size),
      callbacks = callbacks_list,
      verbose = 1)

In [ ]:
# Save the model architecture
with open('fruits_classifier_fine_tuned_vgg16.json', 'w') as f:
    f.write(model.to_json())

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'lower right')
plt.show()

### Classify new images

In [ ]:
from keras.models import load_model
model = load_model('top_model.h5')

In [ ]:
!pwd

In [ ]:
test_image_files_path = "/keras2production/notebooks/1-deeplearning/test_images/"
test_images = !find $test_image_files_path -type f -name "*.jpg"
print(test_images)

In [ ]:
test_images = test_images[0]
print(test_images)

In [ ]:
classes = train_image_array_gen.class_indices
print(classes)

In [ ]:
def classify_image_model(image, classes=classes):
    img = cv2.imread(image)        
    b,g,r = cv2.split(img)       # get b,g,r
    img = cv2.merge([r,g,b])     # switch it to rgb
    plt.imshow(img)
    plt.xticks([]), plt.yticks([])
    plt.show()
    
    image = image_utils.load_img(image, target_size=(img_width, img_height))
    image = image_utils.img_to_array(image)

    image = np.expand_dims(image, axis=0)

    # scale pixels between 0 and 1, sample-wise
    image /= 255.
        
    prediction = model.predict(image)
    
    pred = prediction.argmax()

    for k, v in classes.items():
        if (v == pred):
            pred_label = k
        
    proba = prediction.max()
    
    print("Predicted class: " + pred_label + 
          " with probability " + str(proba*100) + "%")

In [ ]:
classify_image_model(test_images)